Inicialización de los paquetes y librerias necesarios para al aplicación de los algoritmos.

In [ ]:
#Install needed packages (see requirements.txt in github repository)
!pip install numpy
!pip install "scipy==1.4.1"
!pip install Pillow
!pip install cython
!pip install matplotlib
!pip install scikit-image
!pip uninstall keras-nightly
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.15.0
!pip install keras==2.0.8
!pip install opencv-python
!pip uninstall h5py
!pip install h5py==2.10.0
!pip install imgaug

#Download model
!git clone https://github.com/nasca37/Mask_RCNN.git


In [ ]:
%cd Mask_RCNN/

#Download pre-trained weights 
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

In [ ]:
%cd Mask_RCNN/
DATASET = 0

!wget https://www.dropbox.com/s/660s0vgyk83aek9/Img.zip
!wget https://www.dropbox.com/s/52ux32n0yu5xaq0/ImgTrain.zip
!wget https://www.dropbox.com/s/yes1a53053zi6pv/Maskjson.zip
!wget https://www.dropbox.com/s/brkhhd31cytt7z3/MaskTrainjson.zip
!wget https://www.dropbox.com/s/f7hfuw0o36t0lox/ftc.py


!unzip Img.zip
!unzip Maskjson.zip 
!unzip ImgTrain.zip
!unzip MaskTrainjson.zip  

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from matplotlib import pyplot
from csv import reader
import json
import numpy as np
import ntpath
import os
import sys
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from ftc import FTCsegmentation

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.model import MaskRCNN

from mrcnn.utils import Dataset
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

DATASET_DIR_TEST=os.path.join(ROOT_DIR, "Img")
#DATASET_MASK_TEST=os.path.join(ROOT_DIR, "Mask")
DATASET_MASK_TEST=os.path.join(ROOT_DIR, "Maskjson")
DATASET_DIR_TRAIN=os.path.join(ROOT_DIR, "ImgTrain")
#DATASET_MASK_TRAIN=os.path.join(ROOT_DIR, "MaskTrain")
DATASET_MASK_TRAIN=os.path.join(ROOT_DIR, "MaskTrainjson")

Using TensorFlow backend.


In [ ]:
############################################################
#  Configurations
############################################################


class ShadowsConfig(Config):
    """Configuration for training on the   dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "Shadows"
    
    # A GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
#    IMAGES_PER_GPU = 2
    IMAGES_PER_GPU = 1
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + shadow
    
    # Number of training steps per epoch
    #STEPS_PER_EPOCH = 200
    STEPS_PER_EPOCH = 100
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    MINI_MASK_SHAPE = (128, 128)
    MASK_SHAPE = [56,56]
    LOSS_WEIGHTS = {
      "rpn_class_loss":1.,
      "rpn_bbox_loss": 1.,
      "mrcnn_class_loss": 1.,
      "mrcnn_bbox_loss": 1.,
      "mrcnn_mask_loss": 1.
  }

In [ ]:
############################################################
#  Dataset
############################################################

import json
import numpy as np
import skimage
import re
class ShadowDataset(Dataset):

    def load_shadow(self, isTrain = True):
        """Load a subset of the nuclei dataset.

        dataset_dir: Root directory of the dataset
        subset: Subset to load. Either the name of the sub-directory,
                such as stage1_train, stage1_test, ...etc. or, one of:
                * train: stage1_train excluding validation images
                * val: validation images from VAL_IMAGE_IDS
        """
        # Add classes. We have one class.
        # Naming the dataset nucleus, and the class nucleus
        self.add_class("shadow", 1, "shadow")
        if isTrain:
          dataset_dir = DATASET_DIR_TRAIN
        else:
          dataset_dir = DATASET_DIR_TEST

        # Get image ids from directory names
        image_ids = next(os.walk(dataset_dir))[2]
        p = 0.99

        n=len(image_ids)
        all_indices=np.arange(n)
        ntrain=int(p*n)
        np.random.seed(100) #run this line before np.random.choice to choose always the same sequence of values
        fimage_ids=np.random.choice(n, ntrain, replace=False)
        
        # Add images
        for image_id in fimage_ids:
            self.add_image(
                "shadow",
                image_id=image_ids[image_id],
                path=os.path.join(dataset_dir, image_ids[image_id]))

        

    def load_mask(self, image_id,isTrain = True):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        id = re.findall(r'\d+', info['id'])[0]
        # Get mask directory from image path
        if isTrain == False:
          #mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "Mask/mask ({}).json".format(id))
          mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "Maskjson/mask ({}).json".format(id))
        else:
          #mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "MaskTrain/mask ({}).json".format(id))
          mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "MaskTrainjson/mask ({}).json".format(id))
          
        # Read mask info from .json file
        datashadows = json.load(open(mask_dir))
        # create one array for all masks, each on a different channel
        mask = np.zeros([datashadows["height"], datashadows["width"], len(datashadows["regions"])],
                        dtype=np.uint8)
             
        for i, p in enumerate(datashadows["regions"]):
            mask[p['y'], p['x'], i] = 1
        
        
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shadow":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

Es importante que en esta casilla se use la ruta que contiene los pesos entrenados. Como GitHub no permite hacer uso de ficheros con tamaños superiores a los 100Mb se deberá bajar de la url proporcionada en el Readme del repositorio.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DIR = "/content/drive/MyDrive/classifierMaskRcnnLAll2.pt"

Mounted at /content/drive


In [ ]:
###################################
#Evaluation of the trained model
###################################

# evaluate the mask rcnn model on the facesFDDB dataset
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
import math

arrayConResultados = list()

# define the prediction configuration
class PredictionConfig(Config):
    # define the name of the configuration
    NAME = "shadow"
    # number of classes (background + eye + mouth)
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    NUM_CLASSES = 1 + 1
    # simplify GPU config
    GPU_COUNT = 1
    DETECTION_MIN_CONFIDENCE = 0.85
    IMAGES_PER_GPU = 1
    MINI_MASK_SHAPE = (128, 128)
    MASK_SHAPE = [56,56]

# calculate the mAP for a model on a given dataset
def evaluate_model(dataset, model, cfg):
	APs = list()
  
	number = 0
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, _, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		if not math.isnan(AP):
		    APs.append(AP)
		    arrayConResultados.append((image_id,AP))
        
		number = number + 1
		if (number % 100) == 0:
		    print("the number is {}".format(number))
      
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP

# load the train dataset
train_set = ShadowDataset()
train_set.load_shadow(isTrain=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
# load the test dataset
test_set = ShadowDataset()
test_set.load_shadow(isTrain=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))

# create config
cfg = PredictionConfig()
# define the model
model = MaskRCNN(mode='inference', model_dir=DEFAULT_LOGS_DIR, config=cfg)
# load model weights
model.load_weights(DIR, by_name=True)

# evaluate model on training dataset
#train_mAP = evaluate_model(train_set, model, cfg)
#print("Train mAP: %.3f" % train_mAP)
#print(arrayConResultados)

# evaluate model on test dataset
#test_mAP = evaluate_model(test_set, model, cfg)
#print("Test mAP: %.3f" % test_mAP)

In [ ]:
import cv2
def display_instances2(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or visualize.random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = visualize.apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8))
    if auto_show:      
        plt.show()
        plt.close()

In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
def id_image(name):
  finalpath = "/content/Mask_RCNN/ImgTrain/" + name
  for id in train_set.image_ids:
    path = train_set.image_reference(id)
    if path == finalpath:
      return id

## Comienzo del proceso de pruebas

A partir de esta casilla empieza el proceso de pruebas de funcionamiento de los algoritmos de mejora.

En la casila inferior se puede colocar el nombre de las imagenes que queremos hacer pruebas y de esta forma obtener sus mascaras mejoradas. Ademas se añade una imagen con una url exterior para hacer pruebas con imagenes de fuera de la base de datos. Para poder usar esas fotografias es necesario que la imagen tenga una resolución de 1024 x 1024 para evitar problemas de compatibilidad con las predicciones del modelo. 

En la parte inferior del notebook se encuentra una función que realiza todos los algoritmos e intenta eliminar la sombra usando un metodo sencillo que no aporta muy buenos resultados. Este último proceso se deja en caso de querer ver como podría funcionar este porceso de eliminado de sombras.

In [ ]:
# 76 589 807 1723 997 2835
from mrcnn import visualize
path1 = "img (2804).jpg"
path2 = "img (2835).jpg"
path3 = "img (76).jpg"
image_id1 = id_image(path1)
image_id2 = id_image(path2)
image_id3 = id_image(path3)

imagen1, image_meta, gt_class_id, gt_bbox, gt_mask =\
  modellib.load_image_gt(train_set, cfg, 
                          image_id1, use_mini_mask=True)
  
imagen2, image_meta, gt_class_id, gt_bbox, gt_mask =\
modellib.load_image_gt(train_set, cfg, 
                        image_id2, use_mini_mask=True)

imagen3, image_meta, gt_class_id, gt_bbox, gt_mask =\
modellib.load_image_gt(train_set, cfg, 
                        image_id3, use_mini_mask=True)

resultados1 = model.detect([imagen1],verbose=0)[0]
resultados2 = model.detect([imagen2],verbose=0)[0]
resultados3 = model.detect([imagen3],verbose=0)[0]

def load_image_prediction(img_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(img_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

img = !wget -O limpiar.jpg https://drive.google.com/uc?id=1be36EhGZJPDH8zJvnbbRgA8Jd7FWTdHw&export=download
img_path='./limpiar.jpg'
imagen4 = load_image_prediction(img_path)  
# convert pixel values (e.g. center)
scaled_image = mold_image(imagen4, cfg)
# convert image into one sample
sample = expand_dims(scaled_image, 0)
resultados4 = model.detect(sample,verbose=0)[0]

visualize.display_instances(imagen4, resultados4['rois'], resultados4['masks'], resultados4['class_ids'], 
                            train_set.class_names, resultados4['scores'])

## Algoritmos de mejora de la detección

Dentro de este apartado se encuentra el aspecto principal de este notebook. Cada uno de los algoritmos contiene una colección de tres ejemplos. Este notebook hace uso de unos pesos que tengo en mi drive por tanto no se puede ejecutar sin mi cuenta. 

### Algoritmo 1 
Este primer algoritmo tiene como objetivo obtener una matriz bidimensional que contendrá que pixeles pertenecen a una sombra.

- El input de este algoritmo es la lista de las máscaras de la imagen.

- EL output es la matriz bidimensional que contiene un 1 en los pixeles que pertenezcan a una sombra.

In [ ]:
#Algoritmo 1
import matplotlib.pyplot as plt
from mrcnn.utils import resize_mask

def obtenerMascaras(masks):
  N = masks.shape[2]
  
  image = np.zeros((1024,1024))
  for i in range(N):

    image = image + masks[:, :, i].astype(int)

  for i in range(image.shape[0]):
    for j in range(image.shape[1]):
      if(image[i][j] > 0):
        image[i][j] = 1

  return image

### Pruebas del funcionamiento del algoritmo

In [ ]:
from matplotlib.pyplot import figure

plt.rcParams["figure.figsize"] = (25,10)

In [ ]:
def overlay_image(image,mask):
  mask_image = skimage.color.label2rgb(mask,bg_color=(0,0,0),colors=[(0,0,0),(1,0,0)])
  plt.imshow(image,cmap='hsv',interpolation='none')
  plt.imshow(mask_image,cmap='gray',interpolation='none',alpha=0.25)
  plt.show()

In [ ]:
mascaras1 = obtenerMascaras(resultados1['masks'])
mascaras2 = obtenerMascaras(resultados2['masks'])
mascaras3 = obtenerMascaras(resultados3['masks'])
mascaras4 = obtenerMascaras(resultados4['masks'])
#Muestra gráfica
plt.axis('off')
plt.subplot(1,2,1)
plt.imshow(mascaras1)
plt.subplot(1,2,2)
plt.imshow(imagen1)
plt.show()
overlay_image(imagen1,mascaras1)


plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(mascaras2)
plt.subplot(1,2,2)
plt.axis("off")
plt.imshow(imagen2)
plt.show()
overlay_image(imagen2,mascaras2)

plt.axis("off")
plt.imshow(imagen2)
plt.show()

plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(mascaras3)
plt.subplot(1,2,2)
plt.axis("off")
plt.imshow(imagen3)
plt.show()
overlay_image(imagen3,mascaras3)

plt.axis("off")
plt.imshow(imagen3)
plt.show()

plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(mascaras4)
plt.subplot(1,2,2)
plt.axis("off")
plt.imshow(imagen4)
plt.show()
overlay_image(imagen4,mascaras4)


### Algoritmo 2
Algoritmo que recibe el resultado del algoritmo anterior y tiene como objetivo obtener una lista con las componentes conexas. Es decir una lista con los pixeles que pertenecen a una sombra.

- Input: Matriz bidimensional que contiene los pixeles que son y no son sombras.

- Output: Lista de componentes conexas

Para resolver este problema se usa la libreria de scipy la cual contiene operaciones para trabajar con elementos de visión por computacion. Esta libreria devuelve una imagen donde cada cc esta etiquetada. De esta forma con un bucle cuadratico obtenemos en función de este indice cada una de las cc.

In [ ]:
#Algoritmo 2
from scipy import ndimage

def obtenerComponentesConexas(image):
  labels,nb = ndimage.label(image)

  listaComponentes = []
  for k in range(nb):
    cc = []
    for i in range(labels.shape[0]):
      for j in range(labels.shape[1]):
        if labels[i][j] == (k+1):
          cc.append((i,j))
    listaComponentes.append(cc)

  return listaComponentes


In [ ]:
def printCC(image,cc):
  mascara = np.zeros((image.shape[0],image.shape[1]))
  if cc != None:
    for i in range(len(cc)):
      x = cc[i][1]
      y = cc[i][0]
      mascara[y,x] = 1

    overlay_image(image,mascara)

def printMask(image,cc):
  mascara = np.zeros((image.shape[0],image.shape[1]))
  if cc != None:
    for i in range(len(cc)):
      x = cc[i][1]
      y = cc[i][0]
      mascara[y,x] = 1
    plt.axis("off")
    plt.imshow(mascara, cmap='gray',  interpolation='nearest')
    plt.savefig('mask.png')

    plt.show()

### Algoritmo 3
Este algoritmo debe generar un histograma de la intensidad de la componente conexa que recibe.

- Input: Componente conexa.
- Output: Histograma con las intensidades.

Para poder calcular esta itesidad se pasa la imagen que recibimos a una imagen en escala de grises. Para obtener esto se usa una función auxiliar que devuelve una matriz bidemensional de forma que se usa la formula de la intensidad. Esta formula esta ajustada a la escala RGB por tanto se deberia ajustar en función al tipo de distribución de color de la imagen.



In [ ]:
#Algoritmo 3

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.21, 0.72, 0.07])

def histograma(original_image,componenteConexa,show=True):
  histograma = []
  histograma = np.zeros(256)
  intensity =  0
  bins = np.arange(256)
  gray_image = rgb2gray(original_image)
  for i in range(len(componenteConexa)):
    x = componenteConexa[i][0]
    y = componenteConexa[i][1]
    intensity = gray_image[x][y]
    intensity = math.floor(intensity)

    histograma[intensity] = histograma[intensity] + 1

  if show:
    plt.bar(bins,histograma,align="center")
    plt.show()
  
  return histograma



### Algoritmo 4
Este algoritmo debe aplicar el algoritmo de otsu para obtener un valor threshold que permita separar las intensidades que pertenezcan a la sombra y los valores que pertenezcan a la luz.

- Input: Histograma.
- Output: Threshold.



In [ ]:
#Algoritmo 4

def obtain_threshold(histograma):
  hist_norm = np.ravel(histograma)/np.max(histograma)
  Q = np.cumsum(hist_norm)

  bins = np.arange(256)

  fn_min = np.inf
  thresh = -1

  for i in range(0,254):
      p1,p2 = np.hsplit(hist_norm,[i]) # probabilities
      q1,q2 = Q[i],Q[254]-Q[i] # cum sum of classes
      b1,b2 = np.hsplit(bins,[i]) # weights

      # finding means and variances
      m1,m2 = np.sum(p1*b1)/q1, np.sum(p2*b2)/q2
      v1,v2 = np.sum(((b1-m1)**2)*p1)/q1,np.sum(((b2-m2)**2)*p2)/q2

      # calculates the minimization function
      fn = v1*q1 + v2*q2
      if fn < fn_min:
          fn_min = fn
          thresh = i

  return thresh

import statistics

def shadow_addition_treshold(hist):
  intensidades = []
  for i in range(len(hist)):
    for j in range(int(hist[i])):
      intensidades.append(i)
  
  media = np.mean(intensidades)
  stv = 0
  if len(intensidades) > 1:
    stv = statistics.stdev(intensidades)
    
  return (media + stv)

In [ ]:
def algoritmos4_2(h):
  separators = FTCsegmentation(h, 1, 0, 2)  #1, 0, y 2 son parametros fijos del algoritmo
  threshold = -1
  if len(separators) == 3:
      threshold = 1
      
  else:
      print("El histograma es unimodal")

  return threshold

In [ ]:
def pruebaHisto(imagen,mascaras):

  lista_componentes_conexas_imagen =  obtenerComponentesConexas(mascaras)
  thresh_values = []
  for i in range(len(lista_componentes_conexas_imagen)):
    print("Histograma de la componente conexa {}".format(i))
    histo = histograma(imagen,lista_componentes_conexas_imagen[i])
    printCC(imagen,lista_componentes_conexas_imagen[i])
    thresh = obtain_threshold(histo)
    thresh2 = algoritmos4_2(histo)
    
    shadow_addition = shadow_addition_treshold(histo)
    print("El valor de separación de esta componente conexa es: {}".format(thresh))
    thresh_values.append((thresh,shadow_addition,thresh2))
    print("---------------------------------------------------------")
    


  return thresh_values

t1 = pruebaHisto(imagen1,mascaras1)
t2 = pruebaHisto(imagen2,mascaras2)
t3 = pruebaHisto(imagen3,mascaras3)
t4 = pruebaHisto(imagen4,mascaras4)

### Algoritmo 5
Este algoritmo debe aplicar el valor threshold que hemos obtenido anteriormente para eliminar de la componente conexa los valores que esten por debajo del valor.
- Input: Componente conexa, imagen original y el valor de separación.
- Output: Componente conexa con los pixeles con intensidades por debajo del valor de separación.


In [ ]:
#Algoritmo 5

def componenteConexaLimpiaThreshold(cc,image,thresh):
  if thresh == -1:
    return cc
    
  gray_image = rgb2gray(image)
  aux = []
  for k in range(len(cc)):
    x = cc[k][1]
    y = cc[k][0]
    intensity = gray_image[y][x]
    intensity = math.floor(intensity)
    if intensity <= thresh:
      aux.append(cc[k])

  return aux


In [ ]:
def pruebaLimpieza(imagen,mascaras,valores):

  lista_componentes_conexas_imagen =  obtenerComponentesConexas(mascaras)
  cc_limpias = []
  for i in range(len(lista_componentes_conexas_imagen)):
    if valores[i][2] != -1:
      ccLimpia = componenteConexaLimpiaThreshold(lista_componentes_conexas_imagen[i],imagen,valores[i][0])
    else:
      #ccLimpia = lista_componentes_conexas_imagen[i]
      ccLimpia = componenteConexaLimpiaThreshold(lista_componentes_conexas_imagen[i],imagen,valores[i][0])
    
    print("Comparacion mascaras de la cc {}".format(i))
    print("El valor threshold es: {}".format(valores[i][0]))
    printCC(imagen,lista_componentes_conexas_imagen[i])
    printCC(imagen,ccLimpia)
    cc_limpias.append(ccLimpia)
    print("---------------------------------------------------------")
  
  return cc_limpias

cc1L = pruebaLimpieza(imagen1,mascaras1,t1)
cc2L = pruebaLimpieza(imagen2,mascaras2,t2)
cc3L = pruebaLimpieza(imagen3,mascaras3,t3)
cc4L = pruebaLimpieza(imagen4,mascaras4,t4)


### Algoritmo 6
Este algoritmo debe aplicar el valor threshold que hemos obtenido anteriormente y la componenete conexa limpia para añadir a la componente los pixeles cercanos que tengan la intensidad por debajo del threshold.
- Input: Componente conexa limpia , imagen original y el valor de separación.
- Output: Componente conexa con los nuevos pixeles con intensidades por debajo del valor de separación.

In [ ]:
#Algoritmo 6

def mejorarSombra(original_image,cc,tresh,mask):
  if tresh == -1:
    return cc

  pila_pixeles = cc.copy()
  gray_image = rgb2gray(original_image)
  threshold = tresh
  while len(pila_pixeles) != 0:
    
    
    punto_aux = pila_pixeles.pop()
    x = punto_aux[1]
    y = punto_aux[0]
    
    if x + 1 < gray_image.shape[1]:
      if gray_image[y][x+1] < threshold and gray_image[y][x+1] > 0 and mask[y][x+1] != 1:
        pila_pixeles.append((y,x+1))
        cc.append((y,x+1))
        mask[y][x+1] = 1
    if x - 1 > 0:
      if gray_image[y][x-1] < threshold and gray_image[y][x-1] > 0 and mask[y][x-1] != 1:
        pila_pixeles.append((y,x-1))
        cc.append((y,x-1))
        mask[y][x-1] = 1

    if y + 1 < gray_image.shape[0]:
      if gray_image[y+1][x] < threshold and gray_image[y+1][x] > 0 and mask[y+1][x] != 1:
        pila_pixeles.append((y+1,x))
        cc.append((y+1,x))
        mask[y+1][x] = 1
        
    if y - 1 > 0:
      if gray_image[y-1][x] < threshold and gray_image[y-1][x] > 0 and mask[y-1][x] != 1 :
        pila_pixeles.append((y-1,x))
        cc.append((y-1,x))
        mask[y-1][x] = 1



  return cc


### Algoritmo 7
Este algoritmo final debe de generar la matriz bidimensional que contiene la detección de las sombras mejoradas.

- Input: Lista de componentes conexas limpias y ampliadas.

- Output: Matriz bidimensional de la máscara final.
    

In [ ]:
#Algotimo auxiliar
def generarMask(lista_cc):
  mask = np.zeros((1024,1024))
  for cc in lista_cc:
    for i in range(len(cc)):

      x = cc[i][1]
      y = cc[i][0]
      mask[y][x] = 1

  return mask


In [ ]:
def pruebaMejora(imagen,ccLimpias,valores):


  cc_mejoradas = []
  mask = generarMask(ccLimpias)
  plt.imshow(mask)
  plt.show()
  for i in range(len(ccLimpias)):
    printCC(imagen,ccLimpias[i])
    ccMejorada = mejorarSombra(imagen,ccLimpias[i],valores[i][1],mask)
    print("Comparacion mascaras de la cc {}".format(i))
    print("El valor threshold es: {}".format(valores[i][1]))
   
    printCC(imagen,ccMejorada)
    printMask(imagen,ccMejorada)
    cc_mejoradas.append(ccMejorada)
    print("---------------------------------------------------------")
  
  return cc_mejoradas

cc1M = pruebaMejora(imagen1,cc1L,t1)
cc2M = pruebaMejora(imagen2,cc2L,t2)
cc3M = pruebaMejora(imagen3,cc3L,t3)
cc4M = pruebaMejora(imagen4,cc4L,t4)


In [ ]:
def showMask(maskList,image):
  mask = generarMask(maskList)
  
  overlay_image(image,mask)


showMask(cc4M,imagen4)

A partir de esta sección se encuentran los dos algoritmos que se han estado probando para limpiar la sombra, no consideramos que sean buenos pero son representativos de aplicaciones para futura investigación.

In [ ]:
#Algoritmo 8
def insideR(mask,y,x,r=10):
  
  for i in range(y,y+r):
    for j in range(x,x+r):
      if i < 1024 and j < 1024:
        if mask[i][j] == 1:
          return True
  
  return False


def eliminarCC(cc,original_image,r=10):
  plt.imshow(original_image)
  plt.show()
  grey_image = rgb2gray(original_image)
  plt.imshow(grey_image)
  plt.show()
  itsDark = []
  itsLight = []
  #mascara = np.zeros((original_image.shape[0],original_image.shape[1]))
  lumineP = []
  for i in range(len(cc)):
    x = cc[i][1]
    y = cc[i][0]
   # mascara[y][x] = 1
    itsDark.append(grey_image[y][x])
    for k in range(1,r):
      x_1 = x + k
      y_1 = y + k
      if y_1 < 1024 and x_1 < 1024:
        itsLight.append(grey_image[y_1][x_1])
  
  itsLight = list(set(itsLight) - set(itsDark))
  Id = np.mean(itsDark)
  Il = np.mean(itsLight)
  ratio = Il/Id
  print(ratio)
  aux_image = original_image.copy()
  print(len(cc))
  pasos = 0
  for g in range(len(cc)):
    x = cc[g][1]
    y = cc[g][0]
    pixel_value = aux_image[y][x]
    pixel_value= (pixel_value[0]*ratio,pixel_value[1]*ratio,pixel_value[2]*ratio)
    aux_image[y][x] = pixel_value
    pasos = pasos + 1

  return aux_image



import cv2
def eliminarCC2(cc,original_image,r=10):
  #plt.imshow(original_image)
  #plt.show()
  grey_image = rgb2gray(original_image)
  ycc = cv2.cvtColor(original_image,cv2.COLOR_RGB2YCrCb)
  #plt.imshow(grey_image)
  #plt.show()
  itsDark = []
  itsLight = []
  #mascara = np.zeros((original_image.shape[0],original_image.shape[1]))
  lumineP = []
  for i in range(len(cc)):
    x = cc[i][1]
    y = cc[i][0]
   # mascara[y][x] = 1
    itsDark.append(ycc[y][x][0])
    for k in range(1,r):
      x_1 = x + k
      y_1 = y + k
      if y_1 < 1024 and x_1 < 1024:
        itsLight.append(ycc[y_1][x_1][0])
  
  itsLight = list(set(itsLight) - set(itsDark))
  Id = np.mean(itsDark)
  Il = np.mean(itsLight)
  diff = Il - Id
  ratio = Il/Id
  print("El ratio es {} y la diferencia es {}".format(ratio,diff))
  aux_image = ycc.copy()
  
  for g in range(len(cc)):
    x = cc[g][1]
    y = cc[g][0]
    pixel_value = aux_image[y][x]
    pixel_value= (pixel_value[0] + (diff) ,pixel_value[1],pixel_value[2] + ratio)
    aux_image[y][x] = pixel_value
    


  imagen_final = cv2.cvtColor(aux_image,cv2.COLOR_YCR_CB2RGB)

  return imagen_final


In [ ]:
#Conjunto
def mejorarImagen(id_image,mostrar = False):
 
  original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(train_set, cfg, 
                            id_image, use_mini_mask=True)

  # make prediction
  yhat = model.detect([original_image], verbose=0)[0]


  visualize.display_instances(original_image, yhat['rois'], yhat['masks'], yhat['class_ids'], 
                            train_set.class_names, yhat['scores'])

  binary_mask = obtenerMascaras(yhat['masks'])
  plt.imshow(original_image)
  plt.show()
  overlay_image(original_image,binary_mask)
  lista_componentesConexas = obtenerComponentesConexas(binary_mask)
  mask = generarMask(lista_componentesConexas)
  copiaOriginal = original_image.copy()
  mask_final=np.zeros((original_image.shape[0],original_image.shape[1]))

  mask_clean = np.zeros((original_image.shape[0],original_image.shape[1]))
  idx = 0
  
  for cc in lista_componentesConexas:
    hist = histograma(copiaOriginal,cc,show=False)
    threshold = obtain_threshold(hist)
    corrector = shadow_addition_treshold(hist)
    
    cc_clean = componenteConexaLimpiaThreshold(cc,copiaOriginal,threshold)
    #print("Componente limpia")
    #print(cc_clean)
    if cc_clean != None:

      for i in range(len(cc_clean)):
        mask_clean[cc_clean[i][0]][cc_clean[i][1]] = 1

    cc_final = mejorarSombra(copiaOriginal,cc_clean,corrector,mask)
    #print("Componente final")
    #print(cc_final)
    idx = idx + 1

    
    if cc_final != None:

      for i in range(len(cc_final)):
        mask_final[cc_final[i][0]][cc_final[i][1]] = 1
      
      temp_img = eliminarCC2(cc_final,original_image)
      original_image = temp_img
  
  overlay_image(copiaOriginal,mask_clean)
  
  if mostrar:
    overlay_image(copiaOriginal,mask_final)

    plt.imshow(original_image)
    plt.show()
    
    


    

In [ ]:
mejorarImagen(id_image(path1),True)

In [ ]:
mejorarImagen(id_image(path2),True)

In [ ]:
mejorarImagen(id_image(path3),True)